In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Avellaneda-Stoikov Baseline Performance\n",
    "\n",
    "Evaluate the classical market making baseline"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('../python')\n",
    "\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "\n",
    "from env.market_env import MarketMakerEnv\n",
    "from baselines.avellaneda_stoikov import AvellanedaStoikovAgent, evaluate_agent\n",
    "from backtesting.metrics import PerformanceMetrics\n",
    "\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (14, 6)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Test Different Risk Aversion Parameters"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "env = MarketMakerEnv()\n",
    "gammas = [0.01, 0.05, 0.1, 0.5, 1.0]\n",
    "results_by_gamma = {}\n",
    "\n",
    "for gamma in gammas:\n",
    "    print(f\"\\nTesting gamma = {gamma}...\")\n",
    "    agent = AvellanedaStoikovAgent(risk_aversion=gamma)\n",
    "    results = evaluate_agent(agent, env, n_episodes=50)\n",
    "    results_by_gamma[gamma] = results\n",
    "    print(f\"  Mean PnL: ${results['mean_pnl']:.2f}\")\n",
    "    print(f\"  Sharpe: {results['sharpe_ratio']:.3f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot comparison\n",
    "fig, axes = plt.subplots(1, 3, figsize=(18, 5))\n",
    "\n",
    "# Mean PnL\n",
    "mean_pnls = [results_by_gamma[g]['mean_pnl'] for g in gammas]\n",
    "axes[0].bar(range(len(gammas)), mean_pnls, color='steelblue')\n",
    "axes[0].set_xticks(range(len(gammas)))\n",
    "axes[0].set_xticklabels([f'{g}' for g in gammas])\n",
    "axes[0].set_xlabel('Risk Aversion (γ)')\n",
    "axes[0].set_ylabel('Mean PnL ($)')\n",
    "axes[0].set_title('Mean PnL by Risk Aversion', fontweight='bold')\n",
    "axes[0].axhline(y=0, color='r', linestyle='--', alpha=0.5)\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# Sharpe Ratio\n",
    "sharpes = [results_by_gamma[g]['sharpe_ratio'] for g in gammas]\n",
    "axes[1].bar(range(len(gammas)), sharpes, color='orange')\n",
    "axes[1].set_xticks(range(len(gammas)))\n",
    "axes[1].set_xticklabels([f'{g}' for g in gammas])\n",
    "axes[1].set_xlabel('Risk Aversion (γ)')\n",
    "axes[1].set_ylabel('Sharpe Ratio')\n",
    "axes[1].set_title('Sharpe Ratio by Risk Aversion', fontweight='bold')\n",
    "axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.5)\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "# Inventory\n",
    "inventories = [results_by_gamma[g]['mean_inventory'] for g in gammas]\n",
    "axes[2].bar(range(len(gammas)), inventories, color='green')\n",
    "axes[2].set_xticks(range(len(gammas)))\n",
    "axes[2].set_xticklabels([f'{g}' for g in gammas])\n",
    "axes[2].set_xlabel('Risk Aversion (γ)')\n",
    "axes[2].set_ylabel('Avg Inventory')\n",
    "axes[2].set_title('Average Inventory by Risk Aversion', fontweight='bold')\n",
    "axes[2].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Detailed Analysis of Best γ"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Find best gamma\n",
    "best_gamma = max(gammas, key=lambda g: results_by_gamma[g]['mean_pnl'])\n",
    "best_results = results_by_gamma[best_gamma]\n",
    "\n",
    "print(f\"Best γ: {best_gamma}\")\n",
    "print(f\"\\nPerformance Metrics:\")\n",
    "metrics = PerformanceMetrics.get_all_metrics(best_results['episode_pnls'])\n",
    "PerformanceMetrics.print_metrics(metrics, f\"AS Baseline (γ={best_gamma})\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot PnL distribution\n",
    "fig, axes = plt.subplots(1, 2, figsize=(14, 5))\n",
    "\n",
    "# Histogram\n",
    "axes[0].hist(best_results['episode_pnls'], bins=20, alpha=0.7, edgecolor='black')\n",
    "axes[0].axvline(x=0, color='r', linestyle='--', linewidth=2, label='Break-even')\n",
    "axes[0].axvline(x=best_results['mean_pnl'], color='g', linestyle='--', linewidth=2, label=f\"Mean: ${best_results['mean_pnl']:.2f}\")\n",
    "axes[0].set_title(f'AS Baseline PnL Distribution (γ={best_gamma})', fontweight='bold')\n",
    "axes[0].set_xlabel('Final PnL ($)')\n",
    "axes[0].set_ylabel('Frequency')\n",
    "axes[0].legend()\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# Cumulative\n",
    "cumulative = np.cumsum(best_results['episode_pnls'])\n",
    "axes[1].plot(cumulative, linewidth=2)\n",
    "axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.5)\n",
    "axes[1].set_title('Cumulative PnL', fontweight='bold')\n",
    "axes[1].set_xlabel('Episode')\n",
    "axes[1].set_ylabel('Cumulative PnL ($)')\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "- AS baseline provides systematic trading strategy\n",
    "- Risk aversion parameter significantly impacts performance\n",
    "- Best γ = 0.1 (typical for market making)\n",
    "- Challenge: RL agent must beat this baseline!"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.11.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}